In [1]:
from time_series.data_generators import LorenzGenerator
from time_series.time_series_models import KernelRidgeRegression, MovingAverageEstimator
from time_series.kernels import GaussianKernel
from time_series.evaluators.mse_one_step import MeanSquaredError

2025-09-16 09:50:22.810 | INFO     | time_series.config:<module>:13 - PROJ_ROOT path is: /home/james/Repo/PhD Repo/time_series_clustering


In [49]:
import yaml

with open("experiment.yaml", "r") as file:
    config = yaml.safe_load(file)

In [50]:
defintitions_conf = config["definitions"]
experiments_conf = config["experiments"]

In [51]:
definitions = dict()
for k, conf in defintitions_conf.items():
    if k == "datasets":
        datasets = dict()
        for d, v  in conf.items():
            datasets[d] = dict()
            if v["generator"] == "Lorenz":
                datasets[d]["generator"] = LorenzGenerator
            if "parameters" in v:
                datasets[d]["parameters"] = v["parameters"]    
            else:
                datasets[d]["parameters"] = {}
        definitions[k] = datasets

    elif k == "models":
        models = dict()
        for m, v  in conf.items():
            models[m] = dict()
            if v["model"] == "KernelRidgeRegression":
                models[m]["model"] = KernelRidgeRegression
                
            if "parameters" in v:
                models[m]["parameters"] = v["parameters"]
            else:
                models[m]["parameters"] = {}
        definitions[k] = models

    elif k == "kernels":
        kernels = dict()
        for i, v  in conf.items():
            kernels[i] = dict()
            if v["kernel"] == "GaussianKernel":
                kernels[i]["kernel"] = GaussianKernel
                
            if "parameters" in v:
                kernels[i]["parameters"] = v["parameters"]
            else:
                kernels[i]["parameters"] = {}
        definitions[k] = kernels
    
    elif k == "evaluators":
        evaluators = dict()
        for i, v  in conf.items():
            evaluators[i] = dict()
            if v["evaluator"] == "MeanSquaredError":
                evaluators[i]["evaluator"] = MeanSquaredError
                
            if "parameters" in v:
                evaluators[i]["parameters"] = v["parameters"]
            else:
                evaluators[i]["parameters"] = {}
        definitions[k] = evaluators

In [52]:
for experiment_name, experiment in experiments_conf.items():
    # Process datasets
    datasets = experiment["datasets"]
    for dataset_name, dataset_confs in datasets.items():
        dataset_def = definitions["datasets"][dataset_name]
        dataset_params = dataset_def["parameters"]

        if "train_val_test_split" in dataset_confs:
            tvt_split = dataset_confs["train_val_test_split"]
        else:
            tvt_split = [1]
        
        # Process overides
        if "sweeps" in dataset_confs:
            for sweep in dataset_confs["parameters"]:
                params = dict(dataset_def["parameters"])

        print(dataset_name, dataset_confs)

dataset1 {'sweeps': {'sweep_cov': {'noise_cov': {'min': '1e-2', 'max': 1, 'N_steps': 10}}}, 'parameters': {'dt': 0.001}, 'train_val_test_split': [0.6, 0.2, 0.2]}


In [48]:
definitions["datasets"]

{'dataset1': {'generator': time_series.data_generators.lorenz_generator.LorenzGenerator,
  'parameters': {'noise_mean': [0, 0, 0],
   'noise_cov': '1e-2',
   'x0': [10, 10, 10],
   'dt': 0.01,
   'T': 10,
   'rho': 28,
   'sigma': 10,
   'beta': 2.4}}}

In [42]:
dataset_params

{'noise_mean': [0, 0, 0],
 'noise_cov': '1e-2',
 'x0': [10, 10, 10],
 'dt': 0.01,
 'T': 10,
 'rho': 28,
 'sigma': 10,
 'beta': 2.4}

In [ ]:
datasetdata

{'dataset1': {'parameters': {'noise_cov': {'min': '1e-2',
    'max': 1,
    'N_steps': 10},
   'dt': 0.001},
  'train_test_split': [0.6, 0.2, 0.2]}}

In [25]:
datasets

[{'dataset1': {'parameters': {'noise_cov': {'min': '1e-2',
     'max': 1,
     'N_steps': 10},
    'dt': 0.001},
   'train_test_split': [0.6, 0.2, 0.2]}}]

In [ ]:
def parse_configs():
    pass

def load_data():
    pass

def load_model():
    pass

def tune_parameters():
    pass

def train_final_model():
    pass

def evaluate_model():
    pass

def generate_reports():
    pass

In [ ]:
class Experiment:
    pass

    def run(self):
        # Parse configs

        # Load data

        # Load model

        # Tune parameters

        # Train final model

        # Evaluate model

        # Generate reports